In [33]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr

In [19]:
# a sample ascii file
f = open('dataset/regnie_dwd/ra031031', 'r')

In [20]:
# extract the lines
lines=[]
for line in f:
    lines.append(repr(line))
print("number of long in the grid", len(lines[1][2:-3].split("-")))
print("number of lat in the grid", len(lines))
print("last line", lines[-1])

number of long in the grid 611
number of lat in the grid 972
last line '    031031      3353 rus4 D            20041230\n'


In [17]:
# the grid is 611x971 (west/east - northsouth)(long-lat)(x,y)
grid = []
for line in lines[:-1]:# last line ignored because it is related to the time-date
    #print(line)
    grid.append([int(l) for l in  line.replace("-"," ").replace("  "," ")[2:-3].split(" ") if l!=""])

grid =pd.DataFrame(grid)
grid

,0,1,2,3,4,5,6,7,8,9,...,601,602,603,604,605,606,607,608,609,610
0,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
1,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
2,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
3,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
4,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
967,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
968,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
969,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999


In [23]:
# compute the real long/lat for the boundaries of the grid
# x:long , y:lat
xdelta = 1.0 * (1./60.)
ydelta = 1.0 * (1./120.)
# boundaries of the grid
mn =[(0,0),(971,0),(0,971),(971,971)]
real_mn =[]

for pt in mn:

    xmn = ( 6. - 10.*xdelta) + (pt[0] - 1)*xdelta
    ymn = (55. + 10.*ydelta) - (pt[1] - 1)*ydelta
    real_mn.append((xmn,ymn))
    
print(f"long range: {real_mn[0][0]}- {real_mn[1][0]}")
print(f"lat range: {real_mn[0][1]}- {real_mn[2][1]}")

long range: 5.816666666666666- 22.0
lat range: 55.09166666666667- 47.0


In [25]:
# compute the real long/lat for each point of the grid
long =[( 6. - 10.*xdelta) + (pt - 1)*xdelta for pt in range(0,611)]
lat =[(55. + 10.*ydelta) - (pt - 1)*ydelta for pt in range(0,971)]

In [32]:
# the grid with the real long as columns amd lat as rows
real_grid = grid.copy()
real_grid.columns=long
real_grid["lat"]=lat
real_grid = real_grid.set_index("lat")
real_grid.head()

,5.816667,5.833333,5.850000,5.866667,5.883333,5.900000,5.916667,5.933333,5.950000,5.966667,...,15.833333,15.850000,15.866667,15.883333,15.900000,15.916667,15.933333,15.950000,15.966667,15.983333
lat,,,,,,,,,,,,,,,,,,,,,
55.091667,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
55.083333,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
55.075000,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
55.066667,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
55.058333,999,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999


In [34]:
# create nc file to store the prec values 
#dims are the long/lat and the dates, the layers(variables) prec values on each day(assci file)
root_grp = Dataset('prec.nc', 'w', format='NETCDF4')
root_grp.description = 'dims are the long/lat and the dates,(variables) prec values on each day'

# create dimensions
root_grp.createDimension('time', None)
root_grp.createDimension('x', 611)
root_grp.createDimension('y', 971)

# create variables
time = root_grp.createVariable('time', 'f8', ('time',))
x = root_grp.createVariable('x', 'f4', ('x',))
y = root_grp.createVariable('y', 'f4', ('y',))
#each field represents a layer of prec values on a day
field = root_grp.createVariable('field', 'f8', ('time', 'x', 'y',))

#upload data on th dims
x[:] = long
y[:] = lat


In [35]:
# upload the prec values over 5 days as an example
for i in range(5):
    time[i] = i
    field[i,:,:] = file

root_grp.close()

In [37]:
# open the nc file as dataset
ds = xr.open_dataset('prec.nc')
# index by value
#ds['temp'].sel(lon=65.5, lat=29.5)
# or index by position
#ds['temp'].isel(lon=5, lat=2)

In [38]:
# extract the prec at a random bgr station at the nearest long/lat
bgr = 13.643653, 53.416793
prec = float(ds.field.sel(x=bgr[0], y=bgr[1]  , time=2, method='nearest').values)
prec

48.0

In [39]:
type(prec)

float

#### extract all the dates in t s and put them in a list, then build nc file for all these days as layers, the we  slice over all days(time) at the same loc to create a t s, and store it in dict with its bgr_id

## apply the func on a df for all bgr stations